In [1]:
# %load DynPartitioner.py
from networkit import *
import math, sys

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "


In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt

In [4]:
def continuousPartition(G, X, tolerance, k=0, isCharged={}):
    # validate input
    n = G.numberOfNodes()
    if len(isCharged) > 0:
        assert(len(isCharged)==n)
        if k > 0:
            assert(sum(isCharged) <= k)
    else:
        isCharged = [False for i in range(n)]
    assert(X > 0)
    assert(tolerance >= 0)
    if (n % X > 0 or (k>0 and k*X != n)) and tolerance == 0:
        if (k > 0):
            print("Creating", k, "partitions of size", X, "from a graph with", n, " nodes and tolerance 0 is impossible.")
        else:
            print("Creating partitions of size", X, "from a graph with", n, " nodes and tolerance 0 is impossible.")
        print("Set tolerance to 1.")
        tolerance = 1
        
    if k > 0:
        assert(n <= (X + tolerance)*k)
        assert(n >= (X - tolerance)*k)
        
    maxNumberOfPartitions = int(n / max(X - tolerance, 1)) if (k == 0) else k
    
    def w(i):
        """
        Weight of cutting after node i.
        TODO: consider other weights beside the single cut edge
        """
        assert(i >= 0)
        assert(i < n)
        if (i == n-1):
            return 0
        else:
            return G.weight(i,i+1)
    
    # allocate cut and predecessor table
    table = [[float("inf") for j in range(maxNumberOfPartitions)] for i in range(n)]
    pred = [[-1 for j in range(maxNumberOfPartitions)] for i in range(n)]
    
    # fill table 
    for i in range(n):
        for j in range(maxNumberOfPartitions):
            if (j == 0):
                if abs(i+1-X) <= tolerance:
                    table[i][j] = w(i)
            elif (i >= (X-tolerance)):
                windowStart = max(i-(X+tolerance),0)
                
                # make sure that no two charged nodes are in the same partition
                chargeEncountered = False
                for l in reversed(range(windowStart, i+1)):
                    assert(l >= windowStart)
                    if isCharged[l]:
                        if chargeEncountered:
                            windowStart = l
                            break
                        else:
                            chargeEncountered = True
                
                predList = [table[l][j-1] for l in range(windowStart, min(i,i-(X-tolerance)+1))]
                if (len(predList) > 0):
                    minPred = min(predList)
                    table[i][j] = minPred + w(i)
                    pred[i][j] = predList.index(minPred) + windowStart
                    
    #for i in range(n):
    #    row = table[i]
    #    print(i, ': [',end="")
    #    for elem in row:
    #        if math.isinf(elem):
    #            print("inf, ", end="")
    #        else:
    #            print(int(elem*100)/100, ", ", end="")
    #    print("]")
                    
    # get result from table
    resultlist = [table[n-1][j] for j in range(maxNumberOfPartitions)]
    if len(resultlist) == 0:
        raise ValueError("Combination of parameters allows no partition!")
    
    # if k is given, select best solution with k subsets. If not, select best overall solution
    if (k == 0):
        bestCutValue = min(resultlist)
        k = resultlist.index(bestCutValue) + 1
    else:
        bestCutValue = table[n-1][k-1]
        
    if (bestCutValue == float("inf")):
        raise ValueError("Combination of parameters allows no partition!")
    result = partitioning.Partition(n)
    result.setUpperBound(k)
    
    # search best path backwards
    j = k-1
    i = n-1
    c = bestCutValue
    
    while (j > 0):
        nextI = pred[i][j]
        assert(nextI >= 0)
        # assign partitions to nodes
        for l in range(nextI+1, i+1):
            result[l] = j
        j -= 1
        c -=w(i)
        i = nextI
        
    # assign partitions to first nodes not covered by previous loop
    for l in range(0, nextI+1):
        result[l] = 0
        
    # check results:
    for i in range(n):
        assert(result[i] >= 0)
        assert(result[i] < k)
        
    for size in result.subsetSizes():
        if (abs(size-X) > tolerance):
            print("For n=", n, ", k=", k, ", X=", X, ", tolerance=", tolerance, ", ", size, " is wrong.")
        assert(abs(size-X) <= tolerance)
    
    return result

In [5]:
def spiralLayout(G, k, rowheight = 10, colwidth = 10):
    n = G.numberOfNodes()
    z = G.upperNodeIdBound()
    x = [0 for i in range(z)]
    y = [0 for i in range(z)]
    for i in range(z):
        if G.hasNode(i):
            if int(i / k) % 2 > 0:
                x[i] = colwidth*(k-(i % k)-1)
            else:
                x[i] = colwidth*(i % k)
            
            y[i] = rowheight*int(i / k)
            
            # adapt coordinates for rounded bends
            
            ydelta = int(rowheight / 4)
            xdelta = colwidth*(1-math.cos(math.pi/3))
            rightwards = int(i / k) % 2 == 0
    
            if i % k == k-1:
                y[i] += ydelta
                x[i] = x[i] - xdelta if rightwards else x[i] + xdelta
            if i > 0 and i % k == 0:
                y[i] -= ydelta
                x[i] = x[i] - xdelta if not rightwards else x[i] + xdelta
        
    for i in range(z):
        x[i] += 1# gephi ignores coordinates with value 0
        y[i] += 1
    return x, y

In [6]:
def naivePartition(G, X):
    n = G.numberOfNodes()
    naivePart = partitioning.Partition(n)
    naivePart.allToSingletons()
    for i in range(n):
        naivePart.moveToSubset(int(i/X), i)
    return naivePart

In [7]:
def mlPartition(G, k, imbalance):
    mlp = partitioning.MultiLevelPartitioner(G, k, imbalance, False, {})
    mlp.run()
    return mlp.getPartition()

In [50]:
def greedyPartition(G, sizelimit):
    n = G.numberOfNodes()
    part = partitioning.Partition(n)
    part.allToSingletons()
    
    def getWeight(edge):
        return G.weight(edge[0], edge[1])
    
    sortedEdges = sorted(G.edges(), key=getWeight)
    
    # merge heaviest edge, as long as allowed
    while len(sortedEdges) > 0:
        heaviestEdge = sortedEdges.pop()
        firstPart = part.subsetOf(heaviestEdge[0])
        secondPart = part.subsetOf(heaviestEdge[1])
        sizeMap = part.subsetSizeMap()
        allowed = True
        if sizeMap[firstPart] + sizeMap[secondPart] > sizelimit:
            allowed = False
        partSet = {firstPart, secondPart}
        for i in range(n-1):
            if part[i] in partSet and part[i+2] in partSet and not part[i+1] in partSet:
                allowed = False #otherwise, would create single embedded node
        if allowed:
            part.mergeSubsets(firstPart, secondPart)
    
    return part

In [51]:
part = greedyPartition(G, 13)

In [17]:
sortedEdges = sorted(edges, key=getWeight)


In [ ]:
filename = sys.argv[1]
X = int(sys.argv[2])
tolerance = int(sys.argv[3])
k = int(sys.argv[4]) if len(sys.argv) >= 5 else 0
chargedNodes = []
for i in range(5, len(sys.argv)):
    chargedNodes.append(int(sys.argv[i]))
    
G = readGraph(filename, Format.METIS)
n = G.numberOfNodes()
isCharged = [False for i in range(n)]
for c in chargedNodes:
    assert(c < n)
    isCharged[c] = True

part = continuousPartition(G, X, tolerance, k, isCharged)
for a in chargedNodes:
	for b in chargedNodes:
		if a != b:
			assert(part[a] != part[b])

for i in range(n):
	print(part[i])
#community.PartitionWriter().write(part, filename+'.part')

In [8]:
G = readGraph('ubiquitin.graph', Format.METIS)
n = G.numberOfNodes()

In [ ]:
isCharged = [False for i in range(n)]
X = 10
tolerance = 3
k = 8
part = continuousPartition(G, X, tolerance, k, isCharged)

In [22]:
part = mlPartition(G, 8, 0.3)

In [ ]:
dynErrors = []
naiveErrors = []

xlist = list(range(4,21))

for X in xlist:
    isCharged = [False for i in range(n)]
    part = continuousPartition(G, X, 3, 0, isCharged)
    error = partitioning.computeEdgeCut(part, G)
    naivePart = naivePartition(G, X)
    naiveError = partitioning.computeEdgeCut(naivePart, G)
    dynErrors.append(error)
    naiveErrors.append(naiveError)

In [ ]:
plt.plot(xlist, dynErrors, xlist, naiveErrors)

In [ ]:
plt.plot?

In [28]:
xcoords, ycoords = spiralLayout(G, 10, 20)

In [44]:
client = gephi.streaming.GephiStreamingClient()
client.clearGraph()
client.exportGraph(G)

In [52]:
client.exportNodeValues(G, part, "partition")

In [46]:
client.exportNodeValues(G, xcoords, 'x')

In [47]:
client.exportNodeValues(G, [-elem for elem in ycoords], 'y')

In [53]:
partitioning.computeEdgeCut(part, G)

6.677457999999999

In [27]:
def exportToGephi(G, xcoords, ycoords, part):
    client = gephi.streaming.GephiStreamingClient()
    client.clearGraph()
    client.exportGraph(G)
    client.exportNodeValues(G, part, "partition")
    client.exportNodeValues(G, xcoords, 'x')
    client.exportNodeValues(G, [-elem for elem in ycoords], 'y')

In [ ]:
part = community.detectCommunities(G)

In [ ]:
part = community.detectCommunities

In [ ]:
community.PLM?

In [ ]:
G = Graph(20)

In [ ]:
for i in range(20-1):
    G.addEdge(i,i+1)

In [ ]:
G.addEdge(0,19)
G.addEdge(1,18)
G.addEdge(2,17)
G.addEdge(3,16)

In [ ]:
n = G.numberOfNodes()

In [ ]:
part = partitioning.Partition(20)
part.allToSingletons()

In [ ]:
for i in range(4):
    part.moveToSubset(0, i)
    part.moveToSubset(0, n-i-1)

In [ ]:
for i in range(n):
    part.moveToSubset(1, i)

In [54]:
partitioning.inspectPartitions(part, G)

-------------------  ---------
# partitions         12
min partition size    1
max partition size   13
avg. partition size   6.33333
imbalance             1.85714
edge cut              6.67746
edge cut (portion)    0.036892
modularity            0.232637
-------------------  ---------


In [55]:
part.compact()

In [57]:
for i in range(n):
    print(i, part[i])

0 0
1 1
2 1
3 1
4 2
5 2
6 2
7 3
8 3
9 2
10 2
11 2
12 1
13 1
14 1
15 4
16 5
17 6
18 7
19 7
20 6
21 6
22 5
23 8
24 6
25 5
26 5
27 8
28 1
29 3
30 5
31 8
32 1
33 1
34 5
35 3
36 9
37 5
38 5
39 3
40 3
41 3
42 2
43 2
44 2
45 10
46 10
47 2
48 3
49 3
50 5
51 5
52 6
53 6
54 6
55 5
56 7
57 6
58 5
59 5
60 1
61 1
62 0
63 0
64 1
65 1
66 2
67 2
68 2
69 3
70 3
71 3
72 3
73 11
74 11
75 11
